In [11]:
import numpy as np
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from os.path import exists
import spacy

In [12]:
language = input('in what language you want to test program', )
if language == 'ru':
    print('вы выбрали русский язык')
    nlp = spacy.load("ru_core_news_md")
else: 
    print('English language has been choosed')
    nlp = spacy.load("en_core_web_md") 

English language has been choosed


In [13]:

#Предобработка текста
def preprocess(text):
    text = text.lower().strip() #приводим к нижнему регистру и избавляеся от лишних пробелов
    text = re.sub(r'<.*?>', '', text)#убираем htlm таги
    text = re.sub(r'[^\w\s]', ' ', text) #убираем пунктицию и символы как ',", # итд
    text = re.sub(r'\s+', ' ', text)# убираем доп пробелы
    return text

#Приводит слово к изначальному типу
def lemmatizer(string):
    a = nlp(string)
    a = [word.lemma_ for word in a]
    return " ".join(a)

In [14]:
def get_cosine_similarity(matrix_1, matrix_2 = None):
    #Случай если мы хотим сравнить с матрицу саму с собой
    if matrix_2 == None:
        matrix_2 = matrix_1
    #переводим разреженную матрицу в заполненную матрицу
    matrix_1 = matrix_1.todense()
    matrix_2 = matrix_2.todense()
    similarity = np.dot(matrix_1, matrix_2.T)
    norm_matrix_1 = np.linalg.norm(matrix_1, axis=1)
    norm_matrix_2 = np.linalg.norm(matrix_2, axis=1)
    similarity = np.squeeze(np.asarray(similarity))
    cosine = similarity / (norm_matrix_1 * norm_matrix_2)
    return cosine

In [15]:
#Считываем Датасет если нету, то используем тестовый.
if exists('./data/test.csv'):
    df = pd.read_csv('./data/test.csv')
    df['new_text'] = df.apply(lambda x: lemmatizer(preprocess(x['Text'])), axis=1)
else:
    text_1 = "This is a foo bar sentence ."
    text_2 = "This sentence is similar to a foo bar sentence ."
    text_3  = "What is this string ? Totally not related to the other two lines ."
    df = [text_1, text_2, text_3]

In [26]:
vectorizer = TfidfVectorizer()
transformed = vectorizer.fit_transform( df['new_text'])
#Если мы ничего не передаем, то сравнение будет с самим собой
test = input('enter sentence you want to compare', )
if test != '':
    test_processed = lemmatizer(preprocess(test))
    test_vector = vectorizer.transform([test_processed])
    print(test_processed)
else:
    test_vector = transformed

an omelette be a dish make from egg fry with butter or oil in a fry pan


In [27]:
cosine = get_cosine_similarity(transformed, test_vector)
print("Cosine similarity created by me:", cosine)
print("Cosine similarity from sklearn:", cosine_similarity(transformed, test_vector))

Cosine similarity created by me: [0.03348681 0.02229539 0.01943495 0.10653829 0.03171482 0.
 0.         0.         0.         0.2389625  0.52260575 0.07680578
 0.02677069 0.04112583 0.03073994 0.01419474]
Cosine similarity from sklearn: [[0.03348681]
 [0.02229539]
 [0.01943495]
 [0.10653829]
 [0.03171482]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.2389625 ]
 [0.52260575]
 [0.07680578]
 [0.02677069]
 [0.04112583]
 [0.03073994]
 [0.01419474]]


In [28]:
max_index = np.unravel_index(np.argmax(cosine), cosine.shape)
print('similarity:', np.max(cosine))
print('test string have the highest similarity with this sentence: \n', df['Text'][max_index[0]])

similarity: 0.5226057530528185
test string have the highest similarity with this sentence: 
 A pancake (or hotcake, griddlecake, or flapjack) is a flat cake, often thin and round, prepared from a starch-based batter that may contain eggs, milk and butter and cooked on a hot surface such as a griddle or frying pan, often frying with oil or butter.
